## Using Virtual Tables

In [1]:
import sqlite3
import sqlite_vec

db_path = "/Users/abhinav/Desktop/shit/virtual_tables.db"
db = sqlite3.connect(db_path)
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)


In [2]:
# Create table without any embeddings column
# Create virtual tables for storing embeddings with vec0
db.execute(
    """
    CREATE TABLE goods (
        row_id INT PRIMARY KEY,
        Schedule VARCHAR(10),
        Serial_No INT,
        Chapter_Heading_Subheading_Tariff_Item TEXT,
        Description_of_Goods TEXT,
        CGST_Rate DECIMAL(5, 2),
        SGST_UTGST_Rate DECIMAL(5, 2),
        IGST_Rate DECIMAL(5, 2),
        Compensation_Cess VARCHAR(10)
    );
    """
)

In [3]:
import pandas as pd
from models import Row

df = pd.read_excel("/Users/abhinav/Desktop/shit/data/goods.xlsx")
df = df.where(pd.notnull(df), None)

# Iterate through all the rows and convert them into Rows
# Each row is considered as a tuple
rows = []
for row in df.itertuples(name=None):
    row = list(row)
    row[0] += 1
    if row[4] is None or row[4] == "":
        row[4] = "N/A"
    rows.append(Row(values=row))

In [4]:
from indexer import SQLiteVecIndexer

indexer = SQLiteVecIndexer(db_path, using_virtual_table=True)
indexer.insert_rows(rows)

In [3]:
from retriever import SQLiteVecRetriever

query = "dried mango slice"
retriever = SQLiteVecRetriever(db_path=db_path, using_virtual_table=True)
results = retriever.retrieve(query, limit=5)
for i in results:
    print(i)

(33, 'I', '30A', '0804', 'Mangoes sliced, dried', '2.5%', '2.5%', '5%', None)
(1670, 'Nil rate', '57A', '0813', 'Tamarind dried', 0, 0, 0, None)
(353, 'II', 16, '0804', 'Dates (soft or hard), figs, pineapples, avocados, guavas ,mangoes (other than mangoes sliced, dried) and mangosteens, dried.', '6%', '6%', '12%', None)
(1663, 'Nil rate', 51, '0804', 'Dates, figs, pineapples, avocados, guavas, mangoes and  mangosteens, fresh.', 0, 0, 0, None)
(355, 'II', 17, '0813', 'Fruit, dried, other than that of headings 0801 to 0806; mixtures of nuts or dried  fruits of Chapter 8 [other than dried tamarind and dried chestnut (singhada)  whether or not shelled or peeled', '6%', '6%', '12%', None)


## Using Normal Tables (Manually)

In [4]:
import sqlite3
import sqlite_vec

db_path = "/Users/abhinav/Desktop/shit/normal_tables.db"
db = sqlite3.connect(db_path)
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [5]:
# Create table with embeddings column of type BLOB to store embeddings
db.execute(
    """
    CREATE TABLE goods (
        row_id INT PRIMARY KEY,
        Schedule VARCHAR(10),
        Serial_No INT,
        Chapter_Heading_Subheading_Tariff_Item TEXT,
        Description_of_Goods TEXT,
        CGST_Rate DECIMAL(5, 2),
        SGST_UTGST_Rate DECIMAL(5, 2),
        IGST_Rate DECIMAL(5, 2),
        Compensation_Cess VARCHAR(10),
        embeddings float[1536]
        check(
            typeof(embeddings) == 'blob'
            and vec_length(embeddings) == 1536
        )
    );
    """
)

In [6]:
import pandas as pd
from models import Row

df = pd.read_excel("/Users/abhinav/Desktop/shit/data/goods.xlsx")
df = df.where(pd.notnull(df), None)

# Iterate through all the rows and convert them into Rows
# Each row is considered as a tuple
rows = []
for row in df.itertuples(name=None):
    row = list(row)
    row[0] += 1
    if row[4] is None or row[4] == "":
        row[4] = "N/A"
    rows.append(Row(values=row))

In [7]:
from indexer import SQLiteVecIndexer

indexer = SQLiteVecIndexer(db_path, using_virtual_table=False)
indexer.insert_rows(rows)

In [8]:
from retriever import SQLiteVecRetriever

query = "dried mango slices"
retriever = SQLiteVecRetriever(db_path, using_virtual_table=False)
results = retriever.retrieve(query)
for i in results:
    print(i)

(33, 'I', '30A', '0804', 'Mangoes sliced, dried', '2.5%', '2.5%', '5%', None)
(1670, 'Nil rate', '57A', '0813', 'Tamarind dried', 0, 0, 0, None)
(353, 'II', 16, '0804', 'Dates (soft or hard), figs, pineapples, avocados, guavas ,mangoes (other than mangoes sliced, dried) and mangosteens, dried.', '6%', '6%', '12%', None)
